### Link to repository: https://github.com/annikaljy/02467/

## Contributions:
### Byun, Jane - Part 1
### Heo, Jiwon - Part 2
### Law, Annika Jie Yu - Part 3

# Assigment 2
### 02467 Computational Social Science Group 6

## Part 1: Properties of the real-world network of Computational Social Scientists

1. Random Network

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import ast
import numpy as np

# 1. Load data
papers_df = pd.read_csv("papers.csv")
author_df = pd.read_excel("author_data.csv")

# 2. Convert stringified lists into actual Python lists
papers_df["author_ids"] = papers_df["author_ids"].apply(ast.literal_eval)

# 3. Create the real co-authorship network (G_real)
G_real = nx.Graph()
for row in papers_df.itertuples():
    authors = row.author_ids
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            G_real.add_edge(authors[i], authors[j])

# 4. Compute real network stats
N = G_real.number_of_nodes()
L = G_real.number_of_edges()
p = L / (N * (N - 1) / 2)
avg_k = 2 * L / N

print(f"Real network: N = {N}, L = {L}, p = {p:.6f}, avg_k = = {avg_k:.2f}")

# 5. Visualize the real network
degrees = dict(G_real.degree())
node_sizes_real = [max(10, degrees[n] * 2) for n in G_real.nodes()]
pos_real = nx.spring_layout(G_real, seed=42, iterations=20)

plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G_real, pos_real, node_size=node_sizes_real, node_color='orange', alpha=0.8)
nx.draw_networkx_edges(G_real, pos_real, edge_color='gray', width=0.8, alpha=0.3)
plt.title("Real Co-authorship Network", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()


# 6. Generate random network using np.random.uniform
def generate_random_network_with_uniform(N, p, seed=None):
    if seed is not None:
        np.random.seed(seed)

    G = nx.Graph()
    G.add_nodes_from(range(N))

    for i in range(N):
        for j in range(i + 1, N):
            if np.random.uniform(0, 1) < p:
                G.add_edge(i, j)
    return G

G_random = generate_random_network_with_uniform(N, p, seed=42)

# 7. Visualize the random network
degrees_rand = dict(G_random.degree())
node_sizes_rand = [max(10, degrees_rand[n] * 2) for n in G_random.nodes()]
pos_rand = nx.spring_layout(G_random, seed=42, iterations=20)

plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G_random, pos_rand, node_size=node_sizes_rand, node_color='lightblue', alpha=0.8)
nx.draw_networkx_edges(G_random, pos_rand, edge_color='gray', width=0.8, alpha=0.3)
plt.title("Random Network", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

### 1. What regime does your random network fall into? Is it above or below the critical threshold?
The random network is above the critical threshold. The critical value for a giant component to appear is roughly $p_c = \frac{1}{N}$, which in our case is about $0.0000658$. Since our $p \approx 0.000419$, it’s well above that. So we’re in the regime where a large connected component is expected to exist.

### 2. According to the textbook, what does the network’s structure resemble in this regime?
In this regime, the network typically forms one big connected cluster along with some small isolated nodes. The degree distribution tends to follow a Poisson pattern, meaning most nodes have degrees close to the average, and high-degree nodes are rare. Overall, the structure looks pretty uniform and lacks distinct features.


### 3. Based on your visualizations, identify the key differences between the actual and the random networks. Explain whether these differences are consistent with theoretical expectations.
The real co-authorship network is much more clustered and uneven—it has hubs, visible communities, and a lot of local structure. The random network, by contrast, looks more uniform and spread out, without clear groupings. These differences line up with what theory predicts: real social networks often show high clustering and modularity, while random networks don’t capture those social patterns.

In [ ]:
# 1. Get giant component from real network
components_real = list(nx.connected_components(G_real))
largest_cc_real = max(components_real, key=len)
G_real_giant = G_real.subgraph(largest_cc_real).copy()

# 2. Get giant component from random network
components_rand = list(nx.connected_components(G_random))
largest_cc_rand = max(components_rand, key=len)
G_rand_giant = G_random.subgraph(largest_cc_rand).copy()

# 3. Calculate average shortest path lengths
avg_path_real = nx.average_shortest_path_length(G_real_giant)
avg_path_rand = nx.average_shortest_path_length(G_rand_giant)

print(f"Avg shortest path (Real Network): {avg_path_real:.4f}")
print(f"Avg shortest path (Random Network): {avg_path_rand:.4f}")

### 1. Why do we consider the giant component only?

Because average shortest path length is only defined between connected node pairs. If we include isolated components, the metric becomes meaningless (infinite distance). The giant component ensures that all node pairs are reachable.

### 2. Why do we consider unweighted edges?

The goal is to examine the basic topological structure of the network (small-world phenomenon), not edge weights or strengths. Unweighted paths help us measure pure connectivity.

### 3. Does the Computational Social Scientists network exhibit the small-world phenomenon?

Likely yes. If the real network shows a similar or slightly higher average shortest path compared to the random graph (with same N, L), and still shows higher clustering (can be computed if needed), it meets the criteria of a small-world network. These networks have short path lengths like random graphs but higher clustering.

# Part 2 Network Analysis in Computational Social Science (Jiwon Heo)

## Exercise 1: Mixing Patterns and Assortativity



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Read data files
authors_df = pd.read_csv('/content/drive/MyDrive/CSS/final_authors.csv')  # Author metadata
papers_df = pd.read_csv('/content/drive/MyDrive/CSS/final_papers.csv')   # Paper-author relationships

# Extract node attributes (country information)
# Handle missing values: Replace missing country_code with 'Unknown'
node_types = dict(zip(authors_df['id'], authors_df['country_code'].fillna('Unknown')))

# Generate edge list (co-authorship relationships from papers)
edges = []
for paper_id, author_list in zip(papers_df['id'], papers_df['author_ids']):
    # Convert string representation of list to actual list
    authors = eval(author_list) if isinstance(author_list, str) else author_list
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            edges.append((authors[i], authors[j]))  # Add co-authorship edge

In [ ]:
import numpy as np

def calculate_assortativity_coefficient(edges, node_types):
    """
    Calculate the Assortativity Coefficient for a network.

    Parameters:
    - edges: List of tuples representing edges [(node1, node2), ...].
    - node_types: Dictionary mapping nodes to their types {node: type}.

    Returns:
    - assortativity_coefficient: Assortativity Coefficient value (float).
    """
    # Map types to indices
    type_set = set(node_types.values())
    type_indices = {t: i for i, t in enumerate(type_set)}
    num_types = len(type_set)

    # Create Mixing Matrix
    e_matrix = np.zeros((num_types, num_types))
    for node1, node2 in edges:
        if node1 in node_types and node2 in node_types:  # Ensure nodes have types
            type1 = type_indices[node_types[node1]]
            type2 = type_indices[node_types[node2]]
            e_matrix[type1, type2] += 1
            e_matrix[type2, type1] += 1  # For undirected graph

    e_matrix /= e_matrix.sum()  # Normalize to probabilities

    # Calculate row and column sums
    a = e_matrix.sum(axis=1)
    b = e_matrix.sum(axis=0)

    # Calculate Assortativity Coefficient
    trace_e = np.trace(e_matrix)
    sum_ab = np.sum(a * b)
    assortativity_coefficient = (trace_e - sum_ab) / (1 - sum_ab)

    return assortativity_coefficient

# Assortativity Coefficient
assortativity = calculate_assortativity_coefficient(edges, node_types)
print("Assortativity Coefficient:", assortativity)


#### 1. Calculate the Assortativity Coefficient for the network based on the country of each node. Implement the calculation using the formula provided during the lecture, also available in this paper (equation 2, here for directed networks). Do not use the NetworkX implementation.


The assortativity coefficient for the network based on the country of each node has been calculated using the provided formula, yielding a value of approximately 0.3897. This indicates moderate assortativity, suggesting that nodes are somewhat more likely to connect with others from the same country.

## Part 2: Configuration model
In the following, we are going to assess the significance of the assortativity by comparing the network's assortativity coefficient against that of random networks generated through the configuration model.

### 2. Implement the configuration model using the double edge swap algorithm to generate random networks. Ensure each node retains its original degree but with altered connections. Create a function that does that by following these steps:
**a**. Create an exact copy of your original network.

**b**. Select two edges, e1=(u,v) and e2=(x,y), ensuring u != y and v != x.

**c**. Flip the direction of e1
 to e1=(v,u)
 50% of the time. This ensure that your final results is not biased, in case your edges were sorted (they usually are).

**d**. Ensure that new edges e′1=(e1[0],e2[1])
 and e′2=(e2[0],e1[1])
 do not already exist in the network.

**e**. Remove edges e1
 and e2
 and add edges e′1
 and e′2
.

**f**. Repeat steps **b** to **e** until you have performed E*10 swaps, where E is the total number of edges.

In [ ]:
import pandas as pd
import random
from collections import Counter
import numpy as np

# Load the actual data from the provided CSV files
authors_df = pd.read_csv('/content/drive/MyDrive/CSS/final_authors.csv')
papers_df = pd.read_csv('/content/drive/MyDrive/CSS/final_papers.csv')

# Extract node types (author_id -> country_code)
node_types = dict(zip(authors_df['id'], authors_df['country_code'].fillna('Unknown')))

# Extract edges (co-authorship relationships)
edges = []
for _, row in papers_df.iterrows():
    # Convert author_ids from string to list if necessary
    author_ids = eval(row['author_ids']) if isinstance(row['author_ids'], str) else row['author_ids']
    for i in range(len(author_ids)):
        for j in range(i + 1, len(author_ids)):
            edges.append((author_ids[i], author_ids[j]))  # Create undirected edges

In [ ]:
import pandas as pd
import networkx as nx
import random
import time

def double_edge_swap_preserve_degrees(graph, num_swaps):
    """Generate randomized network using double edge swap algorithm"""
    G = graph.copy()
    edges_list = list(G.edges())
    swaps_done = 0
    attempts = 0
    max_attempts = num_swaps * 10

    while swaps_done < num_swaps and attempts < max_attempts:
        attempts += 1

        # Select two distinct edges with non-overlapping nodes
        e1_idx, e2_idx = random.sample(range(len(edges_list)), 2)
        e1 = edges_list[e1_idx]
        e2 = edges_list[e2_idx]

        if len(set(e1 + e2)) < 4:
            continue

        # Randomly flip edge direction
        if random.random() < 0.5:
            e1 = (e1[1], e1[0])

        # Create new edges
        e1_new = (e1[0], e2[1])
        e2_new = (e2[0], e1[1])

        # Validate new edges
        if (G.has_edge(*e1_new) or G.has_edge(*e2_new) or
            e1_new[0] == e1_new[1] or e2_new[0] == e2_new[1]):
            continue

        # Update graph and edge list
        G.remove_edges_from([e1, e2])
        G.add_edges_from([e1_new, e2_new])

        # Maintain edge list integrity
        if e1_idx > e2_idx:
            edges_list.pop(e1_idx)
            edges_list.pop(e2_idx)
        else:
            edges_list.pop(e2_idx)
            edges_list.pop(e1_idx)

        edges_list.extend([e1_new, e2_new])
        swaps_done += 1

    return G

# Load data and build graph
authors_df = pd.read_csv('/content/drive/MyDrive/CSS/final_authors.csv')
papers_df = pd.read_csv('/content/drive/MyDrive/CSS/final_papers.csv')
node_types = dict(zip(authors_df['id'], authors_df['country_code'].fillna('Unknown')))

# Construct collaboration network
G = nx.Graph()
for _, paper in papers_df.iterrows():
    author_ids = eval(paper['author_ids']) if isinstance(paper['author_ids'], str) else paper['author_ids']
    for i in range(len(author_ids)):
        for j in range(i+1, len(author_ids)):
            G.add_edge(author_ids[i], author_ids[j])

# Configuration model implementation
num_swaps = len(G.edges()) * 10
G_random = double_edge_swap_preserve_degrees(G, num_swaps)

# Validation outputs
print(f"Original network: {len(G.nodes)} nodes, {len(G.edges)} edges")
print(f"Randomized network: {len(G_random.nodes)} nodes, {len(G_random.edges)} edges")
print("Degree preservation:", dict(G.degree()) == dict(G_random.degree()))


#### 2. Configuration Model Implementation
The code successfully implements the configuration model using the double edge swap algorithm. It preserves the original degree of each node while randomizing the connections within the network. This is achieved by repeatedly swapping edges between randomly selected pairs, ensuring no self-loops or duplicate edges are introduced during the swaps. The function *double_edge_swap_preserve_degrees* performs these operations effectively.

#### 3. Degree Preservation Validation
The algorithm's correctness is confirmed as the degree distribution of nodes in the original network and the randomized network remains identical after edge swaps. This demonstrates that the configuration model accurately retains node degrees while altering network connections.

### Part 3: Analyzing Assortativity in Random Networks



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def calculate_assortativity_by_country(graph, node_types):
    """Calculate country-based assortativity coefficient"""
    nx.set_node_attributes(graph, node_types, "country_code")
    return nx.attribute_assortativity_coefficient(graph, "country_code")

# Original network validation
original_degrees = dict(G.degree())
swapped_degrees = dict(G_random.degree())
print("Degree distribution consistency:", original_degrees == swapped_degrees)

# Calculate original network assortativity
original_assort = calculate_assortativity_by_country(G, node_types)
print(f"Original network assortativity: {original_assort:.4f}")

# Generate and analyze 100 random networks
num_simulations = 100
random_assort = []
progress_interval = max(1, num_simulations//10)  # Progress update interval

print(f"\nGenerating {num_simulations} random networks...")
start_time = time.time()

for i in range(num_simulations):
    # Create new randomized network
    G_random = double_edge_swap_preserve_degrees(G, num_swaps)

    # Calculate assortativity
    random_assort.append(calculate_assortativity_by_country(G_random, node_types))

    # Progress update
    if (i+1) % progress_interval == 0:
        elapsed = time.time() - start_time
        print(f"Completed {i+1}/{num_simulations} ({elapsed:.1f}s elapsed)")

# Analyze results
mean_assort = np.mean(random_assort)
std_assort = np.std(random_assort)
z_score = (original_assort - mean_assort) / std_assort

print(f"\nResults analysis:")
print(f"- Random networks mean: {mean_assort:.4f}")
print(f"- Random networks std: {std_assort:.4f}")
print(f"- Original network Z-score: {z_score:.2f}")

# Visualization settings
plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(random_assort, bins=20, alpha=0.7,
                           label='Random Networks Distribution')
plt.axvline(original_assort, color='r', linestyle='--',
           label=f'Original Network ({original_assort:.4f})')
plt.axvline(mean_assort, color='g', linestyle=':',
           label=f'Random Mean ({mean_assort:.4f})')

# 95% confidence interval
ci_lower = mean_assort - 1.96*std_assort
ci_upper = mean_assort + 1.96*std_assort
plt.axvspan(ci_lower, ci_upper, color='gray', alpha=0.2,
           label='95% Confidence Interval')

plt.title('Assortativity Coefficient Distribution Comparison', fontsize=14)
plt.xlabel('Assortativity Coefficient', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Statistical significance determination
if original_assort > ci_upper:
    print("\nConclusion: Original network shows significantly higher assortativity than random networks (p < 0.05)")
elif original_assort < ci_lower:
    print("\nConclusion: Original network shows significantly lower assortativity than random networks (p < 0.05)")
else:
    print("\nConclusion: No significant difference between original and random networks")


In [ ]:
import seaborn as sns
import matplotlib.patches as mpatches

# Ensure existing calculation results are available
# (random_assort, original_assort variables assumed to exist)
assert 'random_assort' in globals(), "Existing execution results required"
assert 'original_assort' in globals(), "Original data required"

# Style settings
sns.set_style("whitegrid")
plt.rcParams['font.size'] = 12

# Create plots for detailed analysis
fig, ax = plt.subplots(2, 1, figsize=(12, 10))

# Full distribution
ax[0].hist(random_assort, bins=50, alpha=0.5, color='blue',
          edgecolor='black', density=True)
sns.kdeplot(random_assort, color='red', ax=ax[0], linewidth=2)
ax[0].axvline(original_assort, color='green', linestyle='--', linewidth=2)
ax[0].set_title('Full Distribution Overview')
ax[0].set_xlabel('Assortativity Coefficient')
ax[0].set_ylabel('Density')

# Zoomed-in region (around 0)
zoom_range = (-0.05, 0.05)  # Adjust region of interest
ax[1].hist(random_assort, bins=100, alpha=0.5, color='blue',
         edgecolor='black', density=True, range=zoom_range)
sns.kdeplot(random_assort, color='red', ax=ax[1], linewidth=2)
ax[1].axvline(original_assort, color='green', linestyle='--', linewidth=2)
ax[1].axvline(np.mean(random_assort), color='purple', linestyle=':', linewidth=2)
ax[1].set_xlim(zoom_range)
ax[1].set_title('Zoomed View (Zero-centered Region)')
ax[1].set_xlabel('Assortativity Coefficient')
ax[1].set_ylabel('Density')

# Add statistical annotations
stats_text = f"""Original: {original_assort:.4f}
Random Mean: {np.mean(random_assort):.4f}
95% CI: [{np.percentile(random_assort, 2.5):.4f}, {np.percentile(random_assort, 97.5):.4f}]
Std Dev: {np.std(random_assort):.4f}"""
ax[1].annotate(stats_text, xy=(0.65, 0.7), xycoords='axes fraction',
              bbox=dict(boxstyle="round", fc="white"))

# Create legend
legend_elements = [
    mpatches.Patch(facecolor='blue', alpha=0.5, label='Random Networks'),
    plt.Line2D([0], [0], color='red', lw=2, label='KDE Density'),
    plt.Line2D([0], [0], color='green', linestyle='--', lw=2, label='Original Network'),
    plt.Line2D([0], [0], color='purple', linestyle=':', lw=2, label='Random Mean')
]
fig.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

# Boxplot for additional analysis
plt.figure(figsize=(10, 4))
sns.boxplot(x=random_assort, width=0.3, color='skyblue')
plt.axvline(original_assort, color='red', linestyle='--', linewidth=2)
plt.title('Detailed Spread Analysis')
plt.xlabel('Assortativity Coefficient')
plt.xlim(zoom_range)
plt.grid(True, alpha=0.3)
plt.show()

# Percentile calculation
percentile = 100 * sum(a < original_assort for a in random_assort)/len(random_assort)
print(f"\n[Statistical Details]")
print(f"Original value percentile in random distribution: {percentile:.1f}%")
print(f"Original - Random Mean: {original_assort - np.mean(random_assort):.4f}")
print


#### 4. Generate and analyze at least 100 random networks using the configuration model. For each, calculate the assortativity with respect to the country and plot the distribution of these values. Compare the results with the assortativity of your original network to determine if connections within the same country are significantly higher than chance.


The original network's assortativity coefficient with respect to the country is 0.3413, significantly higher than the mean assortativity coefficient of 0.0003 calculated across 100 randomized networks generated using the configuration model. This indicates that connections within the same country are substantially higher than would be expected by chance, as confirmed by a Z-score of 175.74 and a percentile analysis showing the original network's value lies far above the random distribution.

## Exercise 2: Zachary's karate club


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from community import community_louvain

# Load Zachary's Karate Club graph
G = nx.karate_club_graph()

# Visualize the graph with node colors based on club split
node_colors = ['blue' if G.nodes[node]['club'] == 'Mr. Hi' else 'red' for node in G.nodes]
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=True, node_color=node_colors, node_size=500, font_color='white')
plt.title("Zachary's Karate Club Graph Visualization")
plt.show()

# Function to compute modularity of a graph partitioning
def modularity(graph, partition):
    m = graph.size(weight='weight')  # Total edge weight
    degrees = dict(graph.degree(weight='weight'))
    modularity_value = 0

    for community in partition:
        for u in community:
            for v in community:
                if graph.has_edge(u, v):
                    modularity_value += graph[u][v].get('weight', 1) - (degrees[u] * degrees[v]) / (2 * m)

    return modularity_value / (2 * m)

# Compute modularity for the club split partitioning
club_partition = {
    'Mr. Hi': [node for node in G.nodes if G.nodes[node]['club'] == 'Mr. Hi'],
    'Officer': [node for node in G.nodes if G.nodes[node]['club'] == 'Officer']
}
club_modularity = modularity(G, club_partition.values())
print(f"Club Split Modularity: {club_modularity:.4f}")

# Create 1000 randomized versions of the Karate Club network and compute modularity
num_randomizations = 1000
random_modularities = []

for _ in range(num_randomizations):
    G_random = nx.double_edge_swap(G.copy(), nswap=10 * G.number_of_edges(), max_tries=100 * G.number_of_edges())
    random_modularity = modularity(G_random, club_partition.values())
    random_modularities.append(random_modularity)

# Compute average and standard deviation of modularity for random networks
mean_random_modularity = np.mean(random_modularities)
std_random_modularity = np.std(random_modularities)
print(f"Random Networks Modularity - Mean: {mean_random_modularity:.4f}, Std Dev: {std_random_modularity:.4f}")

# Plot distribution of random modularity vs actual modularity
plt.figure(figsize=(10, 6))
plt.hist(random_modularities, bins=30, alpha=0.7, label='Random Networks Modularity')
plt.axvline(club_modularity, color='red', linestyle='--', linewidth=2, label=f'Actual Club Modularity ({club_modularity:.4f})')
plt.title('Distribution of Random Modularity vs Actual Modularity', fontsize=14)
plt.xlabel('Modularity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Use Louvain algorithm to find communities and compute its modularity
louvain_partition = community_louvain.best_partition(G)
louvain_communities = {}
for node, community_id in louvain_partition.items():
    if community_id not in louvain_communities:
        louvain_communities[community_id] = []
    louvain_communities[community_id].append(node)

louvain_modularity = modularity(G, louvain_communities.values())
print(f"Louvain Partition Modularity: {louvain_modularity:.4f}")

# Create confusion matrix to compare Louvain communities with club split
club_labels = {node: 0 if G.nodes[node]['club'] == 'Mr. Hi' else 1 for node in G.nodes}
num_louvain_communities = len(louvain_communities)
confusion_matrix = np.zeros((num_louvain_communities, 2), dtype=int)

for community_id, nodes in louvain_communities.items():
    for node in nodes:
        club_label = club_labels[node]
        confusion_matrix[community_id, club_label] += 1

print("Confusion Matrix:")
print(confusion_matrix)

#### 1. Graph Visualization
The graph of Zachary's karate club is visualized with nodes colored based on the club split attribute. Nodes belonging to one group are marked in red, while those in the other group are marked in blue. This visualization highlights the community structure within the network.



#### 2. Modularity Function Implementation
A custom function is implemented to calculate the modularity of a graph partitioning using Equation 9.12 from the textbook. The function computes modularity by comparing the actual edge density within partitions to the expected density under a random configuration model, ensuring each node retains its degree.



#### 3. Concept of Modularity
Modularity measures the strength of division of a network into communities. It quantifies how well-connected nodes within the same community are compared to what would be expected in a random network with similar degree distribution. Higher modularity indicates stronger community structure.


#### 4. Modularity of Karate Club Split Partitioning
The modularity of the Karate club split partitioning, computed using the custom function, is approximately 0.3413. This value reflects a strong community structure, indicating that nodes within the same club are well-connected compared to connections across different clubs.

#### 5. Modularity Calculation for 1000 Randomized Networks
Using the double edge swap algorithm, 1000 randomized versions of the Karate Club network were generated. For each randomized network, the modularity of the "club" split was calculated and stored in a list for further analysis.

#### 6. Average and Standard Deviation of Random Network Modularity
The modularity values for the randomized networks have a mean of 0.0003 and a standard deviation of 0.0019, significantly lower than the original network's modularity. This comparison highlights that the original network's community structure is far stronger than would occur by chance.

In [ ]:
from networkx.algorithms.community import girvan_newman

# Apply Girvan-Newman algorithm
gn_hierarchy = girvan_newman(G)
gn_partition = next(gn_hierarchy)  # First split of communities
gn_communities = [list(community) for community in gn_partition]

# Compute modularity for Girvan-Newman partition
gn_modularity = modularity(G, gn_communities)
print(f"Girvan-Newman Partition Modularity: {gn_modularity:.4f}")

# Compare Girvan-Newman communities with Louvain communities
print("Girvan-Newman Communities:", gn_communities)
print("Louvain Communities:", list(louvain_communities.values()))


In [ ]:
# Compute centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# Display top influential nodes based on each metric
top_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
top_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
top_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top Nodes by Degree Centrality:", top_degree)
print("Top Nodes by Betweenness Centrality:", top_betweenness)
print("Top Nodes by Closeness Centrality:", top_closeness)


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics.pairwise import euclidean_distances

# Create adjacency matrix for spectral clustering
adj_matrix = nx.to_numpy_array(G)

# Apply spectral clustering with 2 clusters (expected split)
spectral_clustering = SpectralClustering(n_clusters=2, affinity='precomputed', random_state=42)
spectral_labels = spectral_clustering.fit_predict(adj_matrix)

# Assign nodes to predicted groups
spectral_groups = {i: [] for i in range(2)}
for node, label in enumerate(spectral_labels):
    spectral_groups[label].append(node)

print("Spectral Clustering Predicted Groups:", spectral_groups)
print(f"Node 9 belongs to Group {spectral_labels[9]}")


In [ ]:
import matplotlib.colors as mcolors

# Assign colors for visualization
colors = list(mcolors.TABLEAU_COLORS.values())
louvain_colors = {node: colors[louvain_partition[node] % len(colors)] for node in G.nodes}
gn_colors = {node: colors[int(any(node in community for community in gn_communities))] for node in G.nodes}

plt.figure(figsize=(12, 6))

# Visualize Louvain communities
plt.subplot(1, 2, 1)
nx.draw(G, with_labels=True, node_color=list(louvain_colors.values()), node_size=500, font_color="white")
plt.title("Louvain Communities")

# Visualize Girvan-Newman communities
plt.subplot(1, 2, 2)
nx.draw(G, with_labels=True, node_color=list(gn_colors.values()), node_size=500, font_color="white")
plt.title("Girvan-Newman Communities")

plt.show()


####7. Distribution of Random Modularity and Comparison with Actual Modularity
The distribution of modularity values for 1000 randomized versions of the Karate Club network was plotted as a histogram. The actual modularity of the club split, 0.6261, was marked as a vertical green dashed line on the plot. This visualization demonstrates that the original network's modularity is significantly higher than the modularity values observed in the randomized networks, confirming a stronger community structure in the original network compared to random chance.

####8. Comment on the Figure
The club split is a reasonably good partitioning, as evidenced by its modularity value being significantly higher than those observed in randomized networks. The randomization experiment was conducted to establish a baseline for comparison, demonstrating that the original network's modularity reflects genuine community structure rather than random chance. Preserving node degrees ensures that the randomized networks maintain the same connectivity distribution, isolating modularity differences to the structure of connections rather than degree variations.


####9. Modularity of Louvain Algorithm Communities
Using the Python Louvain algorithm implementation, the modularity of the detected communities was calculated as **0.6261**, which is higher than the modularity of the club split partitioning. This comparison reveals that the Louvain algorithm identifies community structures that are more strongly connected internally than those defined by the club split.

####10. Confusion Matrix Analysis
The confusion matrix comparing Louvain communities with the club split partitioning highlights the correspondence between detected communities and predefined groups. While some overlap exists, discrepancies indicate that the Louvain algorithm captures additional or alternative community structures beyond those defined by the club split. This suggests that community detection algorithms can reveal nuanced relationships not immediately evident from predefined partitions.

## Part 3 - Words that characterize Computational Social Science communities

### 1.1 What does TF stand for?
#### Answer: TF stands for term frequency which measures how often a specific term appears in a set of text document. Higher frequency = higher relevance of word to document's content

### 1.2 What does IDF stand for?
#### Answer: IDF stands for inverse document frequency, reducing weight of common words and increasing weight of 'rarer' words. When a word appears in fewer text documents = more meaningful

### 2.

In [ ]:
import pandas as pd
import requests
from collections import Counter
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
MAX_WORKERS = 8
API_DELAY = 0.1

authors_df = pd.read_csv('data/authors.csv')
author_comm = pd.read_csv('data/author_communities.csv')
abstracts = pd.read_csv('data/abstracts_with_collocations.csv')

# Preprocess Tokens
abstracts['tokens'] = abstracts['collocation_tokens'].map(
    lambda x: eval(x) if isinstance(x, str) else x,
    na_action='ignore'
)
work_to_tokens = {
    work_id.split('/')[-1]: tokens  # Extract 'W...' from URLs
    for work_id, tokens in zip(abstracts['id'], abstracts['tokens'])
}

# Parallel API Fetching
def fetch_author_works(author_url):
    try:
        response = requests.get(f"{author_url}?per-page=200", timeout=10)
        if response.status_code == 200:
            works = [w['id'].split('/')[-1] for w in response.json().get('results', [])]
            return (author_url.split('/')[-1], works)  # Return (author_id, works)
        return (author_url.split('/')[-1], None)
    except Exception: return (author_url.split('/')[-1], None)

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    results = list(tqdm(
        executor.map(fetch_author_works, authors_df['Works API URL']),
        total=len(authors_df)
    ))

author_to_works = {k: v for k, v in results if v is not None}
failed_authors = [k for k, v in results if v is None]

In [ ]:
# Community Processing
valid_works = set(abstracts['id'])
community_tokens = {}
community_token_arrays = {}
community_stats = {}

for comm, group in author_comm.groupby('community'):
    works = [
        work
        for author in group['author_id']
        for work in author_to_works.get(author, [])
        if work in valid_works
    ]

    comm_tokens = [
        tok
        for work in works
        for tok in work_to_tokens.get(work, [])
    ]

    community_tokens[comm] = comm_tokens

# Save
with open('community_tokens.json', 'w') as f:
    json.dump(community_tokens, f)

for community, group in tqdm(author_comm.groupby('community'), desc="Processing communities"):
    token_array = []  # store ALL tokens as a list of strings
    stats = {
        'authors_processed': 0,
        'total_works': 0,
        'works_with_tokens': 0
    }

    for author_id in group['author_id']:
        author_key = f"works?filter=author.id:{author_id.split('/')[-1]}"

        if author_key not in author_to_works: continue

        stats['authors_processed'] += 1

        for work_id in author_to_works[author_key]:
            stats['total_works'] += 1
            if work_id in work_to_tokens:
                tokens = work_to_tokens[work_id]
                if tokens:  # Only add non-empty
                    token_array.extend(tokens)
                    stats['works_with_tokens'] += 1

    community_token_arrays[community] = token_array
    community_stats[community] = stats

# Save
with open('community_token_arrays.json', 'w') as f:
    json.dump(community_token_arrays, f)

print("\nArray created and saved")

### 3. Calculate TF for each word and find the top 5 terms within the top 5 communities

In [ ]:
try:
    author_comm = pd.read_csv('data/author_communities.csv')
    with open('community_token_arrays.json') as f:
        community_token_arrays = json.load(f)
except FileNotFoundError as e: raise

community_sizes = author_comm['community'].value_counts().head(5)
top_communities = community_sizes.index.astype(str).tolist()
print("Top 5 communities by author count:", top_communities)

top_community_tf = {}
for comm in top_communities:
    tokens = community_token_arrays.get(comm, [])
    if not tokens: continue

    tf = Counter(tokens)
    total_terms = sum(tf.values())
    top_terms = [
        (term, count/total_terms)
        for term, count in tf.most_common(5)
    ]
    top_community_tf[comm] = top_terms

print("\nTop Terms:")
for comm in top_communities:
    if str(comm) not in top_community_tf: continue
    term_line = ", ".join([f"{term} ({freq:.3f}, {(freq*100):.1f}%)" for term, freq in top_community_tf[str(comm)]])

    print(f"\nCommunity {comm} (Authors: {author_comm[author_comm['community']==int(comm)].shape[0]}): {term_line}")

In [ ]:
## Next, we calculate IDF for every word.

author_comm = pd.read_csv('data/author_communities.csv')
with open('community_token_arrays.json') as f: community_token_arrays = json.load(f)

top_communities = author_comm['community'].value_counts().head(9).index.astype(str).tolist()

documents = [" ".join(community_token_arrays.get(comm, [])) for comm in top_communities]
tf_results = {}
for i, comm in enumerate(top_communities):
    counter = Counter(documents[i].split())
    total = sum(counter.values())
    tf_results[comm] = [(word, count/total) for word, count in counter.most_common(10)]

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents)
feature_names = vectorizer.get_feature_names_out()

tfidf_results = {}
for i, comm in enumerate(top_communities):
    feature_index = tfidf_matrix[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    tfidf_results[comm] = [(feature_names[idx], score) for idx, score in
                         sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:10]]

top_authors = {}
for comm in top_communities:
    top_authors[comm] = (author_comm[author_comm['community'] == int(comm)]
                        .sort_values('degree', ascending=False)
                        .head(3)[['author_name', 'degree', 'author_id']]
                        .values.tolist())

for comm in top_communities:
    print(f"\nCommunity {comm} ({author_comm[author_comm['community']==int(comm)].shape[0]} authors)")

    # TF terms
    tf_str = " | ".join([f"{word}:{score:.3f}" for word, score in tf_results[comm]])
    print(f"Top 10 TF: {tf_str}")

    # TF-IDF terms
    tfidf_str = " | ".join([f"{word}:{score:.3f}" for word, score in tfidf_results[comm]])
    print(f"Top 10 TF-IDF: {tfidf_str}")

    # Top authors in one line
    authors_str = " | ".join([
        f"{name if pd.notna(name) else f'[{author_id}]'}:{degree}"
        for name, degree, author_id in top_authors[comm]
    ])
    print(f"Top 3 Authors: {authors_str}")

### Describe similarities and differences between the communities.
#### Similarities: most of the communities have word like data, information, and users. There are also common recurring themes related to human-computer interactions.
#### Differences: The focus of each community appears to be quite different. E.g, Community 6 focuses on networks, community 17 focuses on social media.

### Why aren't the TFs not necessarily a good description of the communities?
#### Answer: TFs does not account how some terms may be distinctive to a particular community. The words could be common in other communities as well, such as how "data" could appear in all communities but the "data" may be referring to different focuses and subbject matters. Moreover, rare but significant words can get drowned out by the higher frequency terms. Words like "uncertainty" would be more significant than words like "data", but the word "data" appears much more often.

### What base logarithm did you use? Is that important?
#### Answer: The base logarithm is base e. It is not that important as long as it is consistent with the other calculations.

### Are these 10 words more descriptive of the community? If yes, what is it about IDF that makes the words more informative?
#### Answer: Yes, the IDF makes 'rarer' words stand out more, such as 'reddit' (Community 20) and 'superspreaders' (Community 26) which gives more context as to what the tokens are about, as compared to the TF words which only gives generic words like 'people', 'humans' and 'information' which cannot really give much context.

In [ ]:
author_comm = pd.read_csv('data/author_communities.csv')
with open('community_token_arrays.json') as f:
    community_token_arrays = json.load(f)

top_communities = author_comm['community'].value_counts().head(9).index.astype(str).tolist()
top_authors = {}
for comm in top_communities:
    top_authors[comm] = (author_comm[author_comm['community'] == int(comm)]
                        .sort_values('degree', ascending=False)
                        .head(3)[['author_name', 'degree', 'author_id']]
                        .values.tolist())

for comm in top_communities:
    # Prepare text data
    text = " ".join(community_token_arrays.get(comm, []))

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='viridis',
        max_words=50,
        contour_width=3,
        contour_color='steelblue'
    ).generate(text)

    authors_info = "\n".join([
        f"{i+1}. {name if pd.notna(name) else f'[{author_id}]'} (degree: {degree})"
        for i, (name, degree, author_id) in enumerate(top_authors[comm])
    ])

    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Community {comm} Word Cloud", fontsize=16, pad=20)
    plt.figtext(
        0.5, 0.05,
        f"Top Authors:\n{authors_info}",
        ha="center",
        fontsize=12,
        bbox={"facecolor":"white", "alpha":0.8, "pad":5}
    )

    plt.tight_layout()
    plt.show()

### Comment on your results. What can you conclude on the different sub-communities in Computational Social Science?
#### Answer: The different sub-communities may all have different focuses but they are all related to people in some way, with the presence of words like "human", "user", "individual" and "people".

### Look up online the top author in each community. In light of your search, do your results make sense?
#### Yes, each of the top author in the communities appear to be professors in well-established universities who are credible, which correlates to the higher degree in the community. Furthermore, the words in the wordcloud seems to agree with their field of study. For example, in community 8 wordcloud, the top author Stephan Lewandowsky is a psychologist and the words present in the wordcloud appears to be related - such as 'psychological', 'participant', 'research'.

### Go back to Week 1, Exercise 1. Revise what you wrote on the topics in Computational Social Science. In light of your data-driven analysis, has your understanding of the field changed? How? (max 150 words)
#### My understanding has changed slightly. Based on the results I have gotten, I believe Computational Social Science is highly relevant to humans. It is also not a single field of specific study, but rather a tapestry of different niches. Although there are some overlap (TF), the IDF terms shows the different focuses of each community. I also learnt that it is a highly interdisciplinary field,  as shown by the different words of IDFS - 'advertising', 'social media', 'cultural' and 'superspreaders' to name a few.